## SHIFT COMP LINEAR (m=3) 

[shifts] norm: 0.98051; epsilon: 1.949e-02; chi: 3.169e-02; mu: 3.386e-02; sigma: 1.394e-01; omega: 4.456
[no shifts] norm: 0.92529; epsilon: 7.471e-02; chi: 1.341e+00; mu: 3.155e-02; sigma: 1.464e-01; omega: 0.627


## SHIFT COMP QUADRATIC (m=3)

[shifts] norm: 0.77710; epsilon: 2.229e-01; chi: 1.910e-01; mu: 1.884e-01; sigma: 1.162e-01; omega: 1.392
[no shifts] norm: 0.52529; epsilon: 4.747e-01; chi: 4.270e-01; mu: 2.372e-01; sigma: 1.510e-01; omega: 0.775

## SHIHFT COMP H23 (m=3)

[shifts] norm: 0.95532; epsilon: 4.468e-02; chi: 7.373e-02; mu: 6.836e-02; sigma: 1.775e-01; omega: 2.745
[no shifts] norm: 0.87137; epsilon: 1.286e-01; chi: 1.022e-01; mu: 7.410e-02; sigma: 2.141e-01; omega: 1.927

## RP COMP LINEAR 

[none] norm: 0.95557; epsilon: 4.443e-02; chi: 6.803e-02; mu: 1.325e-01; sigma: 2.230e-01; omega: 2.137
[CL] norm: 0.84674; epsilon: 1.533e-01; chi: 7.984e-02; mu: 1.533e-01; sigma: 2.081e-01; omega: 1.682
[IL] norm: 0.50915; epsilon: 4.909e-01; chi: 2.404e-01; mu: 4.462e-01; sigma: 2.553e-01; omega: 0.698
[both] norm: 0.41788; epsilon: 5.821e-01; chi: 2.260e-01; mu: 4.396e-01; sigma: 1.556e-01; omega: 0.713

## RP COMP QUADRATIC

[none] norm: 0.73046; epsilon: 2.695e-01; chi: 1.638e-01; mu: 2.299e-01; sigma: 1.334e-01; omega: 1.255
[CL] norm: 0.55992; epsilon: 4.401e-01; chi: 1.823e-01; mu: 3.539e-01; sigma: 1.622e-01; omega: 0.878
[IL] norm: 0.57102; epsilon: 4.290e-01; chi: 1.854e+00; mu: 5.770e-01; sigma: 2.527e-01; omega: 0.321
[both] norm: 0.67399; epsilon: 3.260e-01; chi: 3.624e-01; mu: 5.282e-01; sigma: 1.976e-01; omega: 0.707